In [1]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras as extras

In [2]:
param_dic = {
    "host"      : "127.0.0.1",
    "database"  : "taxis",
    "user"      : "postgres",
    "password"  : "Snowdav3"
}

In [3]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print('Connection succesful')
    
    return conn

conn = connect(param_dic)
cursor = conn. cursor()

Connecting to the PostgreSQL database...
Connection succesful


In [4]:
def time_lapse(seconds):
    seconds_in_day = 60 * 60 * 24
    seconds_in_hour = 60 * 60
    seconds_in_minute = 60
    
    days = seconds // seconds_in_day
    hours = (seconds - (days * seconds_in_day)) // seconds_in_hour
    minutes = (seconds - (days * seconds_in_day) - (hours * seconds_in_hour)) // seconds_in_minute
    
    return f"{days} days, {hours} hours, {minutes} minutes"

# Parte 1 - SQL

1. 
    a) ¿Puedes describirnos el procedimiento que has seguido para cargar los tres ficheros?

- Mediante Requests solicito el html vía http del link proporcionado.
- Con BeautifoulSoup filtro por los div por año mediante el tag id="faq{year}". 
- De estos divs seleceiono los tag 'ul' (que contienen los links csv href) para los meses indicados

1.    
    b) ¿Puedes compartirnos la definición de la tabla que estás usando para contestar las
preguntas planteadas?

2. Una vez cargados los tres ficheros en tu base de datos, ¿cuántos registros tiene la
tabla? Compártenos la SQL que has usado.

In [17]:
registros = '''SELECT COUNT(vendorid) AS registros_totales
FROM yellowtaxis;
'''

cursor.execute(registros)
result = cursor.fetchall()[0]
print(result)

(15711654,)


3. Para cada uno de los tres meses con datos ¿podrías decirnos cuál es el trayecto más
corto, más largo y el trayecto medio?, en distancia y en tiempo. Compártenos la
sentencia SQL que has usado.


In [5]:
def execute(month: str, trip_metric, trip_measure):
    
    query = None
    if trip_metric == 'longest':
        if trip_measure == 'time':
            longest_trip_time = f'''
                SELECT vendorid, month, tpep_pickup_datetime, tpep_dropoff_datetime, trip_duration_seconds, trip_duration
                FROM yellowtaxis
                WHERE month = '{month}'
                ORDER BY trip_duration_seconds DESC
                LIMIT 1;
            '''
            query = longest_trip_time
        elif trip_measure == 'distance':
            longest_trip_distance = f'''
                SELECT vendorid, month, tpep_pickup_datetime, tpep_dropoff_datetime,
                trip_duration_seconds, trip_duration, trip_distance
                FROM yellowtaxis
                WHERE month = '{month}'
                ORDER BY trip_distance DESC
                LIMIT 1;
            '''
            query = longest_trip_distance
            
    elif trip_metric == 'shortest':
        if trip_measure == 'time':
            shortest_trip_time = f'''
                SELECT *
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0
                ORDER BY trip_duration_seconds ASC,
                trip_distance ASC
                LIMIT 1;

            '''
            query = shortest_trip_time
            
        elif trip_measure == 'distance':
            shortest_trip_distance = f'''
                SELECT *
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0
                ORDER BY trip_distance ASC,
                trip_duration_seconds ASC
                LIMIT 1;

            '''
            query = shortest_trip_distance
    
    elif trip_metric == 'average':
        if trip_measure == 'time':
            avg_trip_time = f'''
                SELECT AVG(trip_duration_seconds) AS trip_duration_avg
                FROM yellowtaxis
                WHERE month = 'January' AND trip_distance > 0 AND trip_duration_seconds > 0

            '''
            query = avg_trip_time
            
        elif trip_measure == 'distance':
            avg_trip_distance = f'''
                SELECT AVG(trip_distance) AS trip_distance_avg
                FROM yellowtaxis
                WHERE month = 'January' AND trip_distance > 0 AND trip_duration_seconds > 0

            '''
            query =  avg_trip_distance
    
    cursor.execute(query)
    result = cursor.fetchall()[0]
    
    return result

# January

In [6]:
# Longest time
jan_lg_t = execute('January', 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from January 2020 took: {jan_lg_t[-1]}")
print()

# Longest distance

jan_lg_d = execute('January', 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from January 2020 was: {jan_lg_d[-1]} miles")
print()


Longest yellow taxi trip time from January 2020 took: 5 days, 22 hours, 5 minutes


Longest yellow taxi trip distance from January 2020 was: 210240.07 miles



In [7]:
# Shortest time
jan_st_t = execute('January', 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from January 2020 took: {jan_st_t[4]} seconds")
print()

# Shortest distance
jan_st_d = execute('January', 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from January 2020 was: {jan_st_d[7]} miles")
print()


Shortest yellow taxi trip time from January 2020 took: 1 seconds


Shortest yellow taxi trip distance from January 2020 was: 0.01 miles



In [8]:
# Average
jan_avg_t =  execute('January', 'average', 'time')
print()
print(f"Average yellow taxi trip time from January 2020 was: {time_lapse(jan_avg_t[0])}")
print()

# Average distance
jan_avg_d = execute('January', 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from January 2020 was: {jan_avg_d[0]} miles")
print()


Average yellow taxi trip time from January 2020 was: 0 days, 0 hours, 15 minutes


Average yellow taxi trip distance from January 2020 was: 2.96561035652413 miles



# February 

In [9]:
feb_lg_t = execute('February', 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from February 2020 took: {feb_lg_t[-1]}")
print()

# Longest distance

feb_lg_d = execute('February', 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from February 2020 was: {feb_lg_d[-1]} miles")
print()


Longest yellow taxi trip time from February 2020 took: 2 days, 4 hours, 27 minutes


Longest yellow taxi trip distance from February 2020 was: 57051.09 miles



In [10]:
# Shortest time
feb_st_t = execute('February', 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from February 2020 took: {feb_st_t[4]} seconds")
print()

# Shortest distance
feb_st_d = execute('February', 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from February 2020 was: {feb_st_d[7]} miles")
print()


Shortest yellow taxi trip time from February 2020 took: 1 seconds


Shortest yellow taxi trip distance from February 2020 was: 0.01 miles



In [11]:
# Average
feb_avg_t =  execute('February', 'average', 'time')
print()
print(f"Average yellow taxi trip time from February 2020 was: {time_lapse(feb_avg_t[0])}")
print()

# Average distance
feb_avg_d = execute('February', 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from February 2020 was: {feb_avg_d[0]} miles")
print()


Average yellow taxi trip time from February 2020 was: 0 days, 0 hours, 15 minutes


Average yellow taxi trip distance from February 2020 was: 2.965610356524205 miles



# March

In [12]:
mar_lg_t = execute('March', 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from March 2020 took: {mar_lg_t[-1]}")
print()

# Longest distance
mar_lg_d = execute('March', 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from March 2020 was: {mar_lg_d[-1]} miles")
print()


Longest yellow taxi trip time from March 2020 took: 1 days, 6 hours, 14 minutes


Longest yellow taxi trip distance from March 2020 was: 269803.73 miles



In [13]:
# Shortest time
mar_st_t = execute('March', 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from February 2020 took: {jan_st_t[4]} seconds")
print()

# Shortest distance
mar_st_d = execute('March', 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from March 2020 was: {mar_st_d[7]} miles")
print()


Shortest yellow taxi trip time from February 2020 took: 1 seconds


Shortest yellow taxi trip distance from March 2020 was: 0.01 miles



In [14]:
# Average
mar_avg_t =  execute('March', 'average', 'time')
print()
print(f"Average yellow taxi trip time from March 2020 was: {time_lapse(mar_avg_t[0])}")
print()

# Average distance
mar_avg_d = execute('March', 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from March 2020 was: {mar_avg_d[0]} miles")
print()


Average yellow taxi trip time from March 2020 was: 0 days, 0 hours, 15 minutes


Average yellow taxi trip distance from March 2020 was: 2.9656103565242202 miles



4. Para cada uno de los tres meses con datos ¿podrías decirnos cuánta es la variación
porcentual en el número de servicios registrados con respecto al mes anterior? Por
ejemplo, en esta pregunta nos gustaría obtener un resultado así. Compártenos la
sentencia SQL que has usado

In [ ]:
import pandas as pd

df = pd.DataFrame({'month': ['january', 'february']})

df

In [ ]:

df['month'] = pd.to_datetime(df['month'], format="%B").dt.month
df

In [47]:
final_df['month'] = pd.to_datetime(final_df['month'], format="%B").strptime('%B')

AttributeError: 'Series' object has no attribute 'strptime'

In [44]:
df.dtypes
df

,month
0,1900-01-01
1,1900-02-01


In [46]:
df['month'] = datetime.datetime.strptime(df['month'], "%B").strftime('%B').strptime('%B')

TypeError: strptime() argument 1 must be str, not Series

In [45]:
import datetime

month = 'January' 
datetime.datetime.strptime(month, "%B").strftime('%B').strptime('%B')

AttributeError: 'str' object has no attribute 'strptime'

In [ ]:
.strftime('%md')